In [9]:
# import statements
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import isnull, when, count, col,avg
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import FMRegressor
from pyspark.ml import Pipeline
from pyspark.sql.functions import min, max, avg
sc = SparkContext.getOrCreate()
spark = SparkSession(sc) 


In [2]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [6]:
df=spark.read.format('csv').option("header","True").option("inferSchema","True").load('50_Startups_dataset.csv')

In [12]:
df = spark.read.csv("50_Startups_dataset.csv", header=True, inferSchema=True)
df.printSchema()
df.show(5)

root
 |-- _c0: integer (nullable = true)
 |-- R&D Spend: double (nullable = true)
 |-- Administration: double (nullable = true)
 |-- Marketing Spend: double (nullable = true)
 |-- State: string (nullable = true)
 |-- Profit: double (nullable = true)

+---+------------------+------------------+------------------+----------+------------------+
|_c0|         R&D Spend|    Administration|   Marketing Spend|     State|            Profit|
+---+------------------+------------------+------------------+----------+------------------+
|  0|165349.30000000002|          136897.9|471784.19999999995|  New York|         192261.93|
|  1|162597.80000000002|         151377.69|         443898.63|California|         191792.16|
|  2|153441.61000000002|101145.65000000001|407934.63999999996|   Florida|191050.49000000002|
|  3|         144372.51|118671.95000000001|         383199.72|  New York|         182902.09|
|  4|         142107.44| 91391.87000000001|366168.51999999996|   Florida|         166188.04|
+---+

In [10]:
#Minimum, Max and Avf Profit from dataset

df.select(
    min("Profit").alias("Min_Profit"),
    max("Profit").alias("Max_Profit"),
    avg("Profit").alias("Avg_Profit")
).show()

+----------+----------+------------------+
|Min_Profit|Max_Profit|        Avg_Profit|
+----------+----------+------------------+
|   14681.5| 192261.93|112012.73920000001|
+----------+----------+------------------+



In [21]:
#Most startup

df.groupby("State").count().orderBy('count', ascending=[False]).show(1)


+----------+-----+
|     State|count|
+----------+-----+
|California|   17|
+----------+-----+
only showing top 1 row



In [22]:
from pyspark.sql.functions import desc

df.groupBy("State") \
  .count() \
  .orderBy(desc("count")) \
  .show(1)

+----------+-----+
|     State|count|
+----------+-----+
|California|   17|
+----------+-----+
only showing top 1 row



In [ ]:
#(iii) Average Administration cost for startups in Florida

from pyspark.sql.functions import avg

df.filter(df.State == "Florida") \
  .agg(avg("Administration").alias("Avg_Admin_Florida")) \
  .show()

In [27]:
df.filter(df.State=="Florida").agg(avg("Administration").alias("Avg_Florida")).show()

+------------------+
|       Avg_Florida|
+------------------+
|121768.99750000004|
+------------------+



In [28]:
df.filter(df.State=="Florida").select(avg("Administration").alias("Avg_Florida_Cost")).show()

+------------------+
|  Avg_Florida_Cost|
+------------------+
|121768.99750000004|
+------------------+



In [29]:
#(iv) How many startups are located in California?

df.filter(df.State=="California").count()

17

In [31]:
#Out of startups with Profit > 100000, how many spent < 50000 on Marketing?
df.filter((df.Profit>100000) & (df["Marketing Spend"]<50000)).count()

1

In [32]:
df.filter(
    (df.Profit > 100000) &
    (df["Marketing Spend"] < 50000)
).count()

1

In [34]:
#(vi) Remove feature sl no and drop rows with any null values

df.drop("sl no")

DataFrame[_c0: int, R&D Spend: double, Administration: double, Marketing Spend: double, State: string, Profit: double]

In [35]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- R&D Spend: double (nullable = true)
 |-- Administration: double (nullable = true)
 |-- Marketing Spend: double (nullable = true)
 |-- State: string (nullable = true)
 |-- Profit: double (nullable = true)



In [36]:
df.printSchema

<bound method DataFrame.printSchema of DataFrame[_c0: int, R&D Spend: double, Administration: double, Marketing Spend: double, State: string, Profit: double]>

In [37]:
df.show(5)

+---+------------------+------------------+------------------+----------+------------------+
|_c0|         R&D Spend|    Administration|   Marketing Spend|     State|            Profit|
+---+------------------+------------------+------------------+----------+------------------+
|  0|165349.30000000002|          136897.9|471784.19999999995|  New York|         192261.93|
|  1|162597.80000000002|         151377.69|         443898.63|California|         191792.16|
|  2|153441.61000000002|101145.65000000001|407934.63999999996|   Florida|191050.49000000002|
|  3|         144372.51|118671.95000000001|         383199.72|  New York|         182902.09|
|  4|         142107.44| 91391.87000000001|366168.51999999996|   Florida|         166188.04|
+---+------------------+------------------+------------------+----------+------------------+
only showing top 5 rows



In [38]:
#drop na

In [39]:
df = df.dropna()

In [40]:
df.show(5)

+---+------------------+------------------+------------------+----------+------------------+
|_c0|         R&D Spend|    Administration|   Marketing Spend|     State|            Profit|
+---+------------------+------------------+------------------+----------+------------------+
|  0|165349.30000000002|          136897.9|471784.19999999995|  New York|         192261.93|
|  1|162597.80000000002|         151377.69|         443898.63|California|         191792.16|
|  2|153441.61000000002|101145.65000000001|407934.63999999996|   Florida|191050.49000000002|
|  3|         144372.51|118671.95000000001|         383199.72|  New York|         182902.09|
|  4|         142107.44| 91391.87000000001|366168.51999999996|   Florida|         166188.04|
+---+------------------+------------------+------------------+----------+------------------+
only showing top 5 rows



In [41]:
df=df.drop("_c0")

In [42]:
df.show(5)

+------------------+------------------+------------------+----------+------------------+
|         R&D Spend|    Administration|   Marketing Spend|     State|            Profit|
+------------------+------------------+------------------+----------+------------------+
|165349.30000000002|          136897.9|471784.19999999995|  New York|         192261.93|
|162597.80000000002|         151377.69|         443898.63|California|         191792.16|
|153441.61000000002|101145.65000000001|407934.63999999996|   Florida|191050.49000000002|
|         144372.51|118671.95000000001|         383199.72|  New York|         182902.09|
|         142107.44| 91391.87000000001|366168.51999999996|   Florida|         166188.04|
+------------------+------------------+------------------+----------+------------------+
only showing top 5 rows



In [44]:
#Number of record in each column
from pyspark.sql.functions import col, count

df.select([
    count(col(c)).alias(c + "_non_null")
    for c in df.columns
]).show()

+------------------+-----------------------+------------------------+--------------+---------------+
|R&D Spend_non_null|Administration_non_null|Marketing Spend_non_null|State_non_null|Profit_non_null|
+------------------+-----------------------+------------------------+--------------+---------------+
|                50|                     50|                      50|            50|             50|
+------------------+-----------------------+------------------------+--------------+---------------+



In [46]:
#Count null
from pyspark.sql.functions import col, sum

df.select([
    sum(col(c).isNull().cast("int")).alias(c + "_nulls")
    for c in df.columns
]).show()

+---------------+--------------------+---------------------+-----------+------------+
|R&D Spend_nulls|Administration_nulls|Marketing Spend_nulls|State_nulls|Profit_nulls|
+---------------+--------------------+---------------------+-----------+------------+
|              0|                   0|                    0|          0|           0|
+---------------+--------------------+---------------------+-----------+------------+



In [47]:
total = df.count()
print("Total rows in dataframe:", total)

Total rows in dataframe: 50


In [48]:
from pyspark.sql.functions import col, sum, count

total = df.count()

summary = df.select([
    count(col(c)).alias(c + "_non_null")
    for c in df.columns
])

summary_nulls = df.select([
    sum(col(c).isNull().cast("int")).alias(c + "_nulls")
    for c in df.columns
])

summary.show()
summary_nulls.show()

print("Total rows:", total)

+------------------+-----------------------+------------------------+--------------+---------------+
|R&D Spend_non_null|Administration_non_null|Marketing Spend_non_null|State_non_null|Profit_non_null|
+------------------+-----------------------+------------------------+--------------+---------------+
|                50|                     50|                      50|            50|             50|
+------------------+-----------------------+------------------------+--------------+---------------+

+---------------+--------------------+---------------------+-----------+------------+
|R&D Spend_nulls|Administration_nulls|Marketing Spend_nulls|State_nulls|Profit_nulls|
+---------------+--------------------+---------------------+-----------+------------+
|              0|                   0|                    0|          0|           0|
+---------------+--------------------+---------------------+-----------+------------+

Total rows: 50


In [50]:
#(i) Use StringIndexer to convert all string columns to numeric

In [49]:
from pyspark.sql.functions import col
from pyspark.ml.feature import StringIndexer

# find all string-type columns
string_cols = [c for c, t in df.dtypes if t == "string"]

for col_name in string_cols:
    indexer = StringIndexer(inputCol=col_name, outputCol=col_name + "_index")
    df = indexer.fit(df).transform(df)

df.show(5)

# drop original string columns (keep only numeric + indexed)
df = df.drop(*string_cols)
df.printSchema()

+------------------+------------------+------------------+----------+------------------+-----------+
|         R&D Spend|    Administration|   Marketing Spend|     State|            Profit|State_index|
+------------------+------------------+------------------+----------+------------------+-----------+
|165349.30000000002|          136897.9|471784.19999999995|  New York|         192261.93|        1.0|
|162597.80000000002|         151377.69|         443898.63|California|         191792.16|        0.0|
|153441.61000000002|101145.65000000001|407934.63999999996|   Florida|191050.49000000002|        2.0|
|         144372.51|118671.95000000001|         383199.72|  New York|         182902.09|        1.0|
|         142107.44| 91391.87000000001|366168.51999999996|   Florida|         166188.04|        2.0|
+------------------+------------------+------------------+----------+------------------+-----------+
only showing top 5 rows

root
 |-- R&D Spend: double (nullable = true)
 |-- Administration:

In [60]:
#(ii) Use VectorAssembler to create a features column; keep only features and Profit

from pyspark.ml.feature import VectorAssembler

feature_cols = [c for c in df.columns if c != "Profit"]

assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features"
)

final_df = assembler.transform(df).select("features", "Profit")
final_df.show(5)
final_df.printSchema()   # should show 'features' (vector) and 'Profit'

+--------------------+------------------+
|            features|            Profit|
+--------------------+------------------+
|[165349.300000000...|         192261.93|
|[162597.800000000...|         191792.16|
|[153441.610000000...|191050.49000000002|
|[144372.51,118671...|         182902.09|
|[142107.44,91391....|         166188.04|
+--------------------+------------------+
only showing top 5 rows

root
 |-- features: vector (nullable = true)
 |-- Profit: double (nullable = true)



In [61]:
#(iii) Split the data into training and test sets (3:1 ratio)
train_df, test_df = final_df.randomSplit([0.75, 0.25], seed=42)

print("Training rows:", train_df.count())
print("Testing rows:", test_df.count())

Training rows: 33
Testing rows: 17


In [62]:
#(iv) Build a LinearRegression model with default parameters
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol="features", labelCol="Profit")
lr_model = lr.fit(train_df)

print("Coefficients:", lr_model.coefficients)
print("Intercept:", lr_model.intercept)

Coefficients: [0.8045625059339423,-0.02265390732952868,0.017240446450070918,195.21886126982628]
Intercept: 50937.839497552806


In [63]:
#(v) Predict on the test data and calculate MSE
from pyspark.ml.evaluation import RegressionEvaluator

predictions = lr_model.transform(test_df)
predictions.select("features", "Profit", "prediction").show(5)

evaluator_mse = RegressionEvaluator(
    labelCol="Profit",
    predictionCol="prediction",
    metricName="mse"
)

mse = evaluator_mse.evaluate(predictions)
print("Mean Squared Error (MSE) =", mse)

+--------------------+-----------------+-----------------+
|            features|           Profit|       prediction|
+--------------------+-----------------+-----------------+
|[542.15,51743.25,...|         35673.51|50397.06685503073|
|[20229.69,65948.0...|         81229.16|69109.18263896614|
|[23641.03,96189.7...|71498.59000000001|70331.05954329063|
|[27893.0199999999...|77798.93000000001|74686.47328297907|
|[44070.0499999999...|         89949.24|88630.06048431213|
+--------------------+-----------------+-----------------+
only showing top 5 rows

Mean Squared Error (MSE) = 60950469.93518621


In [64]:
#(vi) Calculate RMSE on the test data
evaluator_rmse = RegressionEvaluator(
    labelCol="Profit",
    predictionCol="prediction",
    metricName="rmse"
)

rmse = evaluator_rmse.evaluate(predictions)
print("Root Mean Squared Error (RMSE) =", rmse)

Root Mean Squared Error (RMSE) = 7807.078194509531
